# [의료딥러닝] 2nd Homework (Total 100 points)
---

### 학교: 연세대학교(미래) 학과: 보건의료정보관리학과                   이름: 정연수  학번: 2022240611                    

In [1]:
import numpy as np
import pandas as pd

# **가상 의료보험 데이터를 활용하여 regression 해결하기**

In [2]:
df_insurance = pd.read_csv('https://raw.githubusercontent.com/TA-YSH/DAP/main/lab8/insurance.csv')
df_insurance.head()



,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
df_insurance.shape

(1338, 7)

1. 원하는 방식으로 데이터 전처리를 실시하시오 (전처리를 잘 하면 예측 모델의 성능 향상이 가능합니다)
- 불필요한 features 제외하기
- 결측치 처리하기(data imputation)
- 데이터 범위를 정규화하기(scaling)
- 범주 변수(categorical variables)를 인코딩(encoding)하기 등

2. 원하는 모델을 선언하시오.
    - 참고로 Regression 모델은 정말 많습니다. 아래 문서를 참조하여 데이터에 적절하다고 생각되는 Regression model을 골라서 선언하세요.
    - https://scikit-learn.org/stable/supervised_learning.html


- Dependent variables (DV): "charges"
- Independent variables (IV): "charges" 제외한 모든 변수 (그 중 반드시 모두 사용할 필요는 없음)

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import statsmodels.formula.api as smf
import joblib

# 데이터 전처리
df_insurance.drop(columns=['region'], axis=1, inplace=True)
df_insurance = pd.get_dummies(df_insurance, columns=['sex', 'smoker'], drop_first=True)

# bmi30 새로 정의
def bmi30(row):
    return 1 if row['bmi'] >= 30 else 0

# 특징 추가
df_insurance['bmi30'] = df_insurance.apply(bmi30, axis=1)
df_insurance['age_squared'] = df_insurance['age'] ** 2
df_insurance['smoke_bmi'] = df_insurance['bmi30'] * df_insurance['smoker_yes']

# 데이터 분리 (test size=0.2)
df_insurance_train, df_insurance_test = train_test_split(df_insurance, test_size=0.2, random_state=777)

# 데이터 정규화
scaler = MinMaxScaler()
scaler_filename = "scaler.save"

# 1) Train 데이터 정규화
df_norm_train = df_insurance_train.copy()
df_norm_train_y = df_norm_train.pop('charges')
df_norm_train_x = scaler.fit_transform(df_norm_train)
df_norm_train = pd.DataFrame(df_norm_train_x, columns=df_insurance_train.columns.drop('charges'))
df_norm_train['charges'] = df_norm_train_y.values

# Scaler 저장
joblib.dump(scaler, scaler_filename)

# 2) Test 데이터 정규화
df_norm_test = df_insurance_test.copy()
df_norm_test_y = df_norm_test.pop('charges')
scaler = joblib.load(scaler_filename)
df_norm_test_x = scaler.transform(df_norm_test)
df_norm_test = pd.DataFrame(df_norm_test_x, columns=df_insurance_test.columns.drop('charges'))
df_norm_test['charges'] = df_norm_test_y.values

# 정규화된 데이터프레임에 새로운 특징 추가
df_norm_train['bmi30'] = df_insurance_train['bmi30'].values
df_norm_train['age_squared'] = df_insurance_train['age_squared'].values
df_norm_train['smoke_bmi'] = df_insurance_train['smoke_bmi'].values

df_norm_test['bmi30'] = df_insurance_test['bmi30'].values
df_norm_test['age_squared'] = df_insurance_test['age_squared'].values
df_norm_test['smoke_bmi'] = df_insurance_test['smoke_bmi'].values

# 모델 학습 (OLS)
model_improved_expense = smf.ols("charges ~ age_squared + bmi + bmi30 + smoke_bmi + sex_male + smoker_yes + children", data=df_insurance_train).fit()
# 예측 (OLS)
df_improved_pred_y_ols = model_improved_expense.predict(df_insurance_test)
# 모델 저장 (OLS)
joblib.dump(model_improved_expense, "model_improved_regression.pkl")

# 모델 학습 (랜덤 포레스트)
rf = RandomForestRegressor(random_state=777)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]}
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(df_norm_train.drop(columns='charges'), df_norm_train['charges'])

best_rf = grid_search.best_estimator_

# 예측 (랜덤 포레스트)
df_improved_pred_y_rf = best_rf.predict(df_norm_test.drop(columns='charges'))
# 모델 저장 (랜덤 포레스트)
joblib.dump(best_rf, "model_improved_regression_rf.pkl")


['model_improved_regression_rf.pkl']

In [5]:
# 해당 점수로 채점 예정입니다.

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [6]:
## 모델을 TEST 하는 코드 작성 ##

rmse_rf = np.sqrt(mean_squared_error(df_norm_test['charges'], df_improved_pred_y_rf))
r2_rf = r2_score(df_norm_test['charges'], df_improved_pred_y_rf)

print(f'RMSE (Random Forest): {rmse_rf}')
print(f'R² (Random Forest): {r2_rf}')

RMSE (Random Forest): 4706.715743229648
R² (Random Forest): 0.8595272088001026


### 최종 채점

결과는 모델의 RMSE 수치에 따라 채점될 예정입니다:
채점 기준은 다음과 같으니 참고하시길 바랍니다.

**[채점 기준]**

- 이 문제는 런타임 초기화 후, 처음으로 출력된 점수에 대해서만 평가합니다.
- RMSE 수치가 5200 미만: **90 points (만점)**
- RMSE 수치가 5200 이상, 5900 미만: **70 points**
- RMSE 수치가 5900 이상, 6500 미만: **40 points**
- RMSE 수치가 6500 이상, 8000 미만: **20 points**
- RMSE 수치가 8000 이상, 혹은 **R2_score가 0 미만**: **0 point**

위 결과에서 나온 R2_score 수치의 의미를 설명하시오. **(5 points)**

R2 결정계수 값은 모델이 종속변수의 변동성을 얼마나 잘 설명하는지를 나타내는 지표다. 이는 0~1 사이 값을 가지고, 값이 클수록 모델의 설명력이 높은 것을 의미한다.

위 결과에 나온 R2_score 수치는 약 0.86으로, 1에 가깝기 때문에 모델의 설명력이 높다고 말할 수 있다.



위 결과에서 나온 RMSE 수치의 의미를 설명하시오. **(5points)**

RMSE는 예측값과 실제값의 차이를 측정하는 지표다. RMSE 값이 작을수록 모델의 예측이 실제 데이터와 가깝다는 것을 의미한다.

위 결과의 RMSE 수치는 약 4707 이므로, 해당 모델의 예측값과 실제값 사이의 평균적인 차이는 약 4700 달러이다.

해당 의료보험 데이터의 보험료의 평균을 계산해보면 약 13,270달러가 나온다. 이를 바탕으로 NRMSE을 계산하면 4700/13270= 약 0.35 가 나온다. 평균 보험료의 약 35% 오차를 나타낸다는 것이다. 이는 보험료를 다루는 중요한 문제에서 그렇게 좋은 성능을 나타낸다고 보기 어렵다. 따라서 모델을 더 개선하고 정확도를 높일 필요가 있다.

# #

In [ ]:
# end of file